# Emoji Counting

This notebook is to understand how to resolve the second question to gather emoji counts.

In [153]:
%pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 2.9 MB/s eta 0:00:00-:--:--
Note: you may need to restart the kernel to use updated packages.


In [171]:
import emoji
import json
from pathlib import Path
import re
import duckdb

In [26]:
assets = Path(".").absolute().parent / "data"
tweets_path = assets / "farmers-protest-tweets-2021-2-4.json"

# Q2

Find the top 10 most used emojis

In [131]:
with tweets_path.open("rb") as file:
    for i, row in enumerate(file):
        # This is the first row with an emoji, I'll use it to identify a pattern
        if i == 1:
            data = json.loads(row)
            # String with emojis = \xf0\x9f\x9a\x9c\xf0\x9f\x8c\xbeWE WILL WIN\xf0\x9f\x92\xaa
            # All emojis seem to start with the \xf0\x9f bytes, then the next 2 bytes represent
            # the actual symbol. I'll use that pattern to find all values
            binary_content = data['content'].encode("utf-8")
            pattern = re.compile(rb'(\xf0\x9f.{2})')

            content_emojis = pattern.findall(binary_content)
            print(binary_content.count(b'\xf0\x9f'), "->", [e.decode("utf-8") for e in content_emojis])
            break


3 -> ['🚜', '🌾', '💪']


In [152]:
# Row 45 seems to having more conflicts, lets analyze it...
# Post, seems like colors can also be represented as an emoji itself...
# Edge cases:
#   - \xf0\x9f\x99\x8f\xf0\x9f\x8f\xbd
#   - \xf0\x9f\x91\x87\xf0\x9f\x8f\xbe\xf0\x9f\x91\x87\xf0\x9f\x8f\xbe
#   - \xe2\x9c\x8a\xf0\x9f\x8f\xbe
#     \xe2\x9c\x8a\xf0\x9f\x8f\xbe
with tweets_path.open("rb") as file:
    for i, row in enumerate(file):
        if i == 236:
            data = json.loads(row)
            binary_content = data['content'].encode("utf-8")
            print(binary_content)
            print(binary_content.decode("utf-8"))
            pattern = re.compile(rb'(\xf0\x9f.{2}(?:\xf0\x9f\x8f.|))')

            content_emojis = pattern.findall(binary_content)
            print(binary_content.count(b'\xf0\x9f'), "->", [e.decode("utf-8") for e in content_emojis])
            break

# New Pattern: rb'(\xf0\x9f.{2}(?:\xf0\x9f\x8f.|))'


b'Rest In Power Shaheed\xe2\x9c\x8a\xf0\x9f\x8f\xbe\xe2\x9c\x8a\xf0\x9f\x8f\xbe #FarmersProtest #ModiIgnoringFarmersDeaths #FarmersProtests #ModiDontSellFarmers https://t.co/IUbX2hoE5v'
Rest In Power Shaheed✊🏾✊🏾 #FarmersProtest #ModiIgnoringFarmersDeaths #FarmersProtests #ModiDontSellFarmers https://t.co/IUbX2hoE5v
2 -> ['🏾', '🏾']


In [168]:
# Let's test this...
emoji_counter = {}
with tweets_path.open("rb") as file:
    for i, row in enumerate(file):
        data = json.loads(row)
        binary_content = data['content'].encode("utf-8")
        pattern = re.compile(rb'(\xf0\x9f.{2})')
        expected_count = binary_content.count(b'\xf0\x9f')
        if expected_count == 0:
            continue

        for emoji in pattern.findall(binary_content):
            emoji = emoji.decode("utf-8")
            emoji_counter[emoji] = emoji_counter.get(emoji, 0) + 1

display(emoji_counter)


{'🚜': 2972,
 '🌾': 2363,
 '💪': 947,
 '🤫': 46,
 '🤔': 641,
 '🙄': 219,
 '🇮': 2096,
 '🇳': 2094,
 '👇': 1108,
 '🙏': 7286,
 '🏽': 1218,
 '👍': 834,
 '🏻': 2080,
 '🙌': 215,
 '💜': 24,
 '🤣': 1668,
 '🧑': 69,
 '👩': 100,
 '🌱': 128,
 '🕊': 65,
 '🌍': 103,
 '😱': 54,
 '🏾': 195,
 '💫': 10,
 '🌈': 4,
 '🌼': 32,
 '🔰': 1,
 '😂': 3072,
 '💚': 1040,
 '🇵': 11,
 '🇰': 7,
 '😛': 55,
 '💔': 144,
 '🤨': 62,
 '🥳': 43,
 '🔥': 349,
 '😅': 183,
 '👏': 690,
 '😢': 225,
 '📢': 79,
 '🏼': 857,
 '🥺': 100,
 '🌄': 4,
 '😡': 378,
 '😜': 202,
 '😎': 94,
 '😊': 259,
 '🤯': 24,
 '😒': 48,
 '💯': 173,
 '📸': 34,
 '🤦': 192,
 '📷': 8,
 '💕': 79,
 '👳': 54,
 '😞': 70,
 '😁': 280,
 '🤝': 49,
 '😶': 8,
 '😍': 102,
 '🦁': 25,
 '🌏': 104,
 '🌎': 101,
 '🌳': 44,
 '🌿': 49,
 '💮': 30,
 '🏵': 27,
 '🌺': 44,
 '🌻': 30,
 '🌞': 26,
 '🦋': 20,
 '👌': 232,
 '😃': 85,
 '👞': 27,
 '😆': 172,
 '🇦': 37,
 '🇺': 39,
 '😀': 213,
 '🇸': 31,
 '😻': 6,
 '😽': 4,
 '🤓': 18,
 '🙈': 58,
 '🍞': 19,
 '🥗': 14,
 '🍛': 14,
 '🦹': 2,
 '👊': 51,
 '🕺': 7,
 '😏': 102,
 '😳': 37,
 '🤐': 45,
 '😐': 30,
 '💙': 36,
 '🔁': 7,
 '😭': 164,

In [179]:
# Could not identify a regex pattern... I'll just use a library...
emoji_counter = {}
with tweets_path.open("rb") as file:
    for i, row in enumerate(file):
        data = json.loads(row)
        for token in emoji.analyze(data["content"]):
            _e = token.chars
            emoji_counter[_e] = emoji_counter.get(_e, 0) + 1

count = 0
for _e, _c in sorted(emoji_counter.items(), key = lambda x: x[1], reverse=True):
    print(count, _e, _c)
    count += 1
    if count >= 10:
        break


0 🙏 5049
1 😂 3072
2 🚜 2972
3 🌾 2182
4 🇮🇳 2086
5 🤣 1668
6 ✊ 1651
7 ❤️ 1382
8 🙏🏻 1317
9 💚 1040


# Q3

Find the most influyent accounts

In [205]:
# Pretty straightforward, just get the usernames from mentioned users
aux = {}
with tweets_path.open("rb") as file:
    for i, row in enumerate(file):
        data = json.loads(row)
        if not data["mentionedUsers"]:
            continue
        mentions = [m.get("username") for m in data.get("mentionedUsers")]
        for username in mentions:
            if username not in aux:
                aux[username] = 0
            
            aux[username] += 1

out = []
for u, c in sorted(aux.items(), key = lambda x: x[1], reverse=True):
    out.append((u, c))
    if len(out) >= 10:
        break

display(out)


[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]